# Cognica를 이용한 IMDB 데이터 전문 검색하기

전문 검색(FTS: Full-Text Search)은 입력된 문서를 하나 이상의 분석기를 통해 토큰 단위로 분리하고 분리된 토큰을 색인합니다. 그리고 입력된 질의와 관련도가 높은 문서들을 점수화하여 그 순서에 따라 검색 결과를 출력하는 방법입니다. 일반적으로 키워드로 질의하여 
관련 문서를 찾을 때 사용합니다.

여기서는 IMDB 데이터를 전문 검색을 위한 색인을 생성하고 검색하는 방법에 대해서 다룹니다.

## 준비 사항

### 의존성 설치

이 예제는 Cognica가 설치되어 있고 서버가 동작하고 있음을 가정합니다. 설치 방법은 [Cognica 설치하기](https://docs.cognica.io/install)를 참고하여 설치할 수 있습니다.

동작에 필요한 라이브러리를 설치합니다.


In [1]:
!pip install -U pip
!pip install cognica

### 데이터 준비하기

데이터는 IMDB 데이터를 사용하며 이 파일(`imdb_top1000.csv`)은 예제에 미리 준비되어 있습니다. 이 데이터는 [IMDB Movies Dataset](https://www.kaggle.com/datasets/harshitshankhdhar/imdb-dataset-of-top-1000-movies-and-tv-shows)에서 데이터 대한 내용을 확인할 수 있습니다.

데이터를 입수하면 늘 그렇듯 데이터를 살펴보고 의도에 맞게 가공해야 합니다. 데이터의 칼럼은 하나 이상의 타입을 가질 수 있고 이는 데이터를 검색하고 조회하는데 방해가 될 수 있습니다. 특히 대부분 데이터를 다루는 라이브러리에서 NA(Not a Number) float는 타입이고 이는 불편을 유발하는 경우가 많습니다. 따라서 향후 검색을 편리하도록 하기 위해 몇몇 필드는 타입이 일치하도록 가공합니다. 아래 코드에서는 `read_csv`로 데이터를 로딩하는 과정에서 `converter`를 통해 변환하고 있습니다.

Cognica의 특성상 각 칼럼이 같은 타입을 가지도록 강제하지 않습니다. 이 내용은 [Cognica가 데이터를 다루는 특징](https://docs.cognica.io/user-guide/documentdb/data#cognica%EA%B0%80-%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%A5%BC-%EB%8B%A4%EB%A3%A8%EB%8A%94-%ED%8A%B9%EC%A7%95)에서 참고할 수 있습니다.

In [2]:
import pandas as pd

converters = {
    "Gross": lambda x: int(x.replace(",", "")) if x else 0,
    "Certificate": lambda x: x if x else "",
    "Released_Year": lambda x: 0 if x == "PG" else int(x),
}
df = pd.read_csv("imdb_top_1000.csv", converters=converters)
display(df.dtypes)
df.head()

Poster_Link       object
Series_Title      object
Released_Year      int64
Certificate       object
Runtime           object
Genre             object
IMDB_Rating      float64
Overview          object
Meta_score       float64
Director          object
Star1             object
Star2             object
Star3             object
Star4             object
No_of_Votes        int64
Gross              int64
dtype: object

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,28341469
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,134966411
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,534858444
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,57300000
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,4360000


이제 데이터가 정상적으로 준비되었고 이 데이터를 Cognica에 입력합니다.

## Cognica에 데이터 입력

이제 `create_collection`을 통해 `imdb2`이라는 컬렉션을 생성합니다. 색인 생성을 위한 `indexes`를 정의합니다.

하나씩 살펴보면 다음과 같습니다. `index`를 primary key로 정의합니다.

```python
{"fields": ["index"], "unique": True, "index_type": "kPrimaryKey"},
```

다음은 전문 검색을 위한 색인입니다. 색인의 이름을 `sk_fts`로 정의하고 전문 검색에 사용할 필드를 `fields`로 정의합니다.

```python
"name": "sk_fts",
"fields": ["index", "IMDB_Rating", "Series_Title", "Director", "Overview"],
"index_type": "kFullTextSearchIndex",
```

다음은 각 필드에 대한 분석기를 정의합니다. 특별히 `IMDB_Rating`은 float 타입이기 때문에 `Float64Analyzer`를 사용하고 나머지는 일반적인 토큰 분리를 처리하는 `StandardAnalyzer`로 정의하고 있습니다. 자세한 내용은 [전문 검색 색인 스키마](https://docs.cognica.io/user-guide/documentdb/collection/index-fts)를 통해 참고할 수 있습니다.

```python
"options": {
    "index": {
        "analyzer": {"type": "KeywordAnalyzer"},
        "index_options": "doc_freqs",
    },
    "IMDB_Rating": {
        "analyzer": {"type": "Float64Analyzer"},
        "index_options": "doc_freqs",
    },
    "Series_Title": {
        "analyzer": {"type": "StandardAnalyzer"},
        "index_options": "offsets",
    },
    "Director": {
        "analyzer": {"type": "StandardAnalyzer"},
        "index_options": "offsets",
    },
    "Overview": {
        "analyzer": {"type": "StandardAnalyzer"},
        "index_options": "offsets",
    },
},
```

그리고 `insert` 함수를 통해 pd.dataframe에서 json으로 변환된 데이터를 입력합니다.

In [3]:
from cognica import Channel, DocumentDB

channel = Channel("localhost", 10080)
doc_db = DocumentDB(channel)

indexes = [
    {"fields": ["index"], "unique": True, "index_type": "kPrimaryKey"},
    {
        "name": "sk_fts",
        "fields": ["index", "IMDB_Rating", "Series_Title", "Director", "Overview"],
        "unique": False,
        "index_type": "kFullTextSearchIndex",
        "options": {
            "index": {
                "analyzer": {"type": "KeywordAnalyzer"},
                "index_options": "doc_freqs",
            },
            "IMDB_Rating": {
                "analyzer": {"type": "Float64Analyzer"},
                "index_options": "doc_freqs",
            },
            "Series_Title": {
                "analyzer": {"type": "StandardAnalyzer"},
                "index_options": "offsets",
            },
            "Director": {
                "analyzer": {"type": "StandardAnalyzer"},
                "index_options": "offsets",
            },
            "Overview": {
                "analyzer": {"type": "StandardAnalyzer"},
                "index_options": "offsets",
            },
        },
    },
]

if "imdb2" in doc_db.list_collections():
    doc_db.drop_collection("imdb2")
doc_db.create_collection("imdb2", indexes=indexes)
data = df.reset_index().to_dict(orient="records")
display(data[0])
doc_db.insert("imdb2", data)

{'index': 0,
 'Poster_Link': 'https://m.media-amazon.com/images/M/MV5BMDFkYTc0MGEtZmNhMC00ZDIzLWFmNTEtODM1ZmRlYWMwMWFmXkEyXkFqcGdeQXVyMTMxODk2OTU@._V1_UX67_CR0,0,67,98_AL_.jpg',
 'Series_Title': 'The Shawshank Redemption',
 'Released_Year': 1994,
 'Certificate': 'A',
 'Runtime': '142 min',
 'Genre': 'Drama',
 'IMDB_Rating': 9.3,
 'Overview': 'Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.',
 'Meta_score': 80.0,
 'Director': 'Frank Darabont',
 'Star1': 'Tim Robbins',
 'Star2': 'Morgan Freeman',
 'Star3': 'Bob Gunton',
 'Star4': 'William Sadler',
 'No_of_Votes': 2343110,
 'Gross': 28341469}

## 검색하기

이제 검색할 준비가 되었습니다. 이제 `Series_Title`에서 `crime`이 출현하는 문서를 찾아보도록 하겠습니다. 전문 검색은 `$search` 연산자를 사용하고 `query`에 검색어를 입력합니다. 쿼리에 입력 가능한 검색어 문법은 `AND`, `OR`와 같은 불리언 연산이나 와일드 카드, 근접 검색을 사용할 수 있습니다. 자세한 내용은 [전문 검색](https://docs.cognica.io/user-guide/documentdb/search/fts)에서 살펴볼 수 있습니다.
그리고 `$project`로 전달받을 칼럼과 `$hint`를 통해 `sk_fts` 색인이 선택되도록 유도합니다.

검색 1000개의 레코드 중에 `crime`과 관련한 하나의 문서만 검색 되었습니다.

In [4]:
query = "crime"
project = [
        "Series_Title",
        "Director",
        "Overview",
        "IMDB_Rating",
        {"_meta": "$unnest"}
]
search_query = {
    "$search": {
        "query": f"Series_Title:({query})",
    },
    "$project": project,
    "$hint": "sk_fts"
}

doc_db.find("imdb2", search_query).head(10)

,Director,IMDB_Rating,Overview,Series_Title,_meta.doc_id,_meta.score
0,Woody Allen,7.9,An ophthalmologist's mistress threatens to rev...,Crimes and Misdemeanors,846,3.050069


이제 추가로 문서를 검색하기 위해 검색 대상 필드를 `Overview`로 확장해 보겠습니다. 이제 조금 더 많은 문서가 검색 되었습니다. 

In [5]:
query = "crime"
search_query = {
    "$search": {
        "query": f"Series_Title:({query}) OR Overview:({query})",
    },
    "$project": project,
    "$hint": "sk_fts"
}

doc_db.find("imdb2", search_query).head(10)

,Director,IMDB_Rating,Overview,Series_Title,_meta.doc_id,_meta.score
0,Woody Allen,7.9,An ophthalmologist's mistress threatens to rev...,Crimes and Misdemeanors,846,3.050069
1,Jules Dassin,8.2,"Four men plan a technically perfect crime, but...",Du rififi chez les hommes,561,1.907696
2,François Truffaut,8.1,"A young boy, left without attention, delves in...",Les quatre cents coups,892,1.907696
3,Francis Ford Coppola,9.2,An organized crime dynasty's aging patriarch t...,The Godfather,5,1.789835
4,Christopher Nolan,8.2,"After training with his mentor, Batman begins ...",Batman Begins,166,1.789835
5,Stanley Kramer,8.2,"In 1948, an American court in occupied Germany...",Judgment at Nuremberg,512,1.789835
6,Jeethu Joseph,8.3,A man goes to extreme lengths to save his fami...,Drishyam,595,1.789835
7,Aniruddha Roy Chowdhury,8.1,When three young women are implicated in a cri...,Pink,683,1.736202
8,Francis Ford Coppola,7.6,"Follows Michael Corleone, now in his 60s, as h...",The Godfather: Part III,953,1.736202
9,Nishikant Kamat,8.2,Desperate measures are taken by a man who trie...,Drishyam,253,1.685690


이제 이 문서를 `IMDB_Rating` 기준으로 내림차순 정렬합니다.

```python
"$sort": [{"IMDB_Rating": "desc"}],
```

In [6]:
query = "crime"
search_query = [
    {
        "$search": {
            "query": f"Series_Title:({query}) OR Overview:({query})",
        },
        "$project": project,
        "$hint": "sk_fts",
    },
    {
        "$sort": [{"IMDB_Rating": "desc"}],
    },
]

doc_db.find("imdb2", search_query).head(10)

,Director,IMDB_Rating,Overview,Series_Title,_meta.doc_id,_meta.score
0,Francis Ford Coppola,9.2,An organized crime dynasty's aging patriarch t...,The Godfather,5,1.789835
1,Francis Ford Coppola,9.0,The early life and career of Vito Corleone in ...,The Godfather: Part II,11,1.471618
2,Martin Scorsese,8.7,The story of Henry Hill and his life in the mo...,Goodfellas,51,1.400477
3,Todd Phillips,8.5,"In Gotham City, mentally troubled comedian Art...",Joker,308,1.305791
4,Nadine Labaki,8.4,While serving a five-year sentence for a viole...,Capharnaüm,571,1.685690
5,Jeethu Joseph,8.3,A man goes to extreme lengths to save his fami...,Drishyam,595,1.789835
6,Sriram Raghavan,8.3,A series of mysterious events change the life ...,Andhadhun,590,1.685690
7,Jules Dassin,8.2,"Four men plan a technically perfect crime, but...",Du rififi chez les hommes,561,1.907696
8,Christopher Nolan,8.2,"After training with his mentor, Batman begins ...",Batman Begins,166,1.789835
9,Stanley Kramer,8.2,"In 1948, an American court in occupied Germany...",Judgment at Nuremberg,512,1.789835


추가로 `IMDB_Rating`이 8.5점 이상인 레코드만 검색합니다. 

In [7]:
query = "crime"
search_query = [
    {
        "$search": {
            "query": f"(Series_Title:({query}) OR Overview:({query})) AND IMDB_Rating:>8.5",
        },
        "$project": project,
        "$hint": "sk_fts",
    },
    {
        "$sort": [{"IMDB_Rating": "desc"}],
    },
]

doc_db.find("imdb2", search_query).head(10)

,Director,IMDB_Rating,Overview,Series_Title,_meta.doc_id,_meta.score
0,Francis Ford Coppola,9.2,An organized crime dynasty's aging patriarch t...,The Godfather,5,4.745876
1,Francis Ford Coppola,9.0,The early life and career of Vito Corleone in ...,The Godfather: Part II,11,4.042523
2,Martin Scorsese,8.7,The story of Henry Hill and his life in the mo...,Goodfellas,51,3.765935


이제 조금 다른 `crime`이 아닌 유의어인 `infraction` 입력해 봅니다. 전문 검색을 통한 결괏값이 없음을 확인할 수 있습니다. 다음 [벡터 검색](3_vector_search.ko.ipynb)를 통해 이를 개선하는 방법을 확인할 수 있습니다.

In [8]:
query = "infraction"
search_query = [
    {
        "$search": {
            "query": f"Series_Title:({query}) OR Overview:({query})",
        },
        "$project": project,
        "$hint": "sk_fts",
    },
    {
        "$sort": [{"IMDB_Rating": "desc"}],
    },
]

doc_db.find("imdb2", search_query).head(10)

""
